In [2]:
!pip install pandas

   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   -------- ------------------------------- 2.4/11.6 MB 16.8 MB/s eta 0:00:01
   ------------------ --------------------- 5.5/11.6 MB 16.8 MB/s eta 0:00:01
   ------------------------------------ --- 10.5/11.6 MB 18.7 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 18.2 MB/s eta 0:00:00


In [4]:
import pandas as pd
df = pd.read_parquet('finalized_dataset_forvectorization.parquet')

In [5]:
df

,product_id,merged_text,metadata
0,B079VKKJN7,11 Degrees de los Hombres Playera con Logo Neg...,"{'brand': '11 Degrees', 'color': 'Negro', 'loc..."
1,B079Y9VRKS,Camiseta Eleven Degrees Core TS White M 11 D...,"{'brand': '11 Degrees', 'color': 'Blanco', 'lo..."
2,B07DP4LM9H,11 Degrees de los Hombres Core Pull Over Hoodi...,"{'brand': '11 Degrees', 'color': 'Azul', 'loca..."
3,B07G37B9HP,11 Degrees Poli Panel Track Pant XL Black 11...,"{'brand': '11 Degrees', 'color': None, 'locale..."
4,B07LCTGDHY,11 Degrees Gorra Trucker Negro OSFA Talla únic...,"{'brand': '11 Degrees', 'color': 'Negro (', 'l..."
...,...,...,...
1814919,B09FJYBVKS,スポーツブラ 揺れない ヨガウェア レディース トップス 通気性 ナイトブラ スポーツブラジ...,"{'brand': 'Doworspaw', 'color': 'ベージュ', 'local..."
1814920,B09FTJ3S1N,ブラジャー レディース 補正ブラ Gabrioir 揺れない 大きいサイズ バストアップ美胸...,"{'brand': 'Gabrioir', 'color': 'パープル', 'locale..."
1814921,B09G72LDQZ,レディース ブラジャー 補正ブラ トップス Gabrioir 大きいサイズ 揺れない 脇高設...,"{'brand': 'Gabrioir', 'color': 'ベージュ', 'locale..."
1814922,B09GBGRTPB,ナイトブラ ノンワイヤーブラ ブラジャー レディース Gabrioir 大きいサイズ パット...,"{'brand': 'Gabrioir', 'color': 'パープル', 'locale..."


 # Heavy models which take more time on CPU

## BAAI/bge-3m embedding model

In [2]:
!pip install -U FlagEmbedding peft -q

In [23]:
from FlagEmbedding import BGEM3FlagModel
embedding_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True) 

Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 30102.66it/s]
C:\Users\prade\AppData\Roaming\Python\Python312\site-packages\FlagEmbedding\BGE_M3\modeling.py:335: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issu

In [ ]:
import json
from tqdm import tqdm 
bge3_embedded_json = {}

for p_id,m_t,meta in tqdm(df.values , desc = "Encoding embeddings"):
    bge3_embedded_json[p_id] = embedding_model.encode(m_t , max_length= 1024)['dense_vecs']

with open('bge3_embeddedings.json', 'w') as json_file:
    json.dump(bge3_embedded_json, json_file, allow_nan=True)

## gte-Qwen2-7B-instruct

In [32]:
! pip install flash_attn

Defaulting to user installation because normal site-packages is not writeable
  Using cached flash_attn-2.6.3.tar.gz (2.6 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      fatal: not a git repository (or any of the parent directories): .git
      
      
      torch.__version__  = 2.4.0+cpu
      
      
      C:\Users\prade\AppData\Local\Temp\pip-install-decceb2h\flash-attn_a9af8a29de3c46e5a21d6f061a783bc0\setup.py:95: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\prade\AppData\Local\Temp\pip-install-decceb2h\flash-attn_a9af8a29de3c46e5a21d6f061a783bc0\setup.py", line 179, in <module>
          CUDAExtension(


In [33]:
qwen2_embedded_json =  {}
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("Alibaba-NLP/gte-Qwen2-7B-instruct", trust_remote_code=True)
# In case you want to reduce the maximum length:
embedding_model.max_seq_length = 1024

for p_id,m_t,meta in tqdm(df.values , desc = "Encoding embeddings"):
    qwen2_embedded_json[p_id] = embedding_model.encode(m_t)

with open('qwen2_embeddedings.json', 'w') as json_file:
    json.dump(qwen2_embedded_json, json_file, allow_nan=True)

ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

## bge-multilingual-gemma2

In [34]:
from FlagEmbedding import FlagLLMModel
bge_multilingual_gemma2_json = {}
model = FlagLLMModel('BAAI/bge-multilingual-gemma2', 
                     query_instruction_for_retrieval="Given a web search product document , emebed relevant passages that is retrieved in the futire fo a given query.",
                     use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

for p_id,m_t,meta in tqdm(df.values , desc = "Encoding embeddings"):
    bge_multilingual_gemma2_json[p_id] = model.encode_corpus(m_t)

with open('bge_multilingual_gemma2_json.json', 'w') as json_file:
    json.dump(bge_multilingual_gemma2_json, json_file, allow_nan=True)

c:\ProgramData\anaconda3\envs\amzenv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prade\.cache\huggingface\hub\models--BAAI--bge-multilingual-gemma2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

# Local Run Embeddings

### LaBSE model

In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
model = SentenceTransformer('sentence-transformers/LaBSE' , device = device)
batch_size = 512 
sentences = df["merged_text"].values

all_embeddings = []
for start_idx in tqdm(range(0, len(sentences), batch_size), desc="Processing Batches"):
    # Define the end index of the current batch
    end_idx = min(start_idx + batch_size, len(sentences))
    batch = sentences[start_idx:end_idx]
    embeddings = model.encode(batch , convert_to_numpy= True)
    all_embeddings.append(embeddings)


all_embeddings = np.concatenate(all_embeddings, axis=0)
np.save('LaBSE_embeddings.npy', all_embeddings)

c:\Users\prade\.conda\envs\cudagllm\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\prade\.conda\envs\cudagllm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\prade\.conda\envs\cudagllm\Lib\site-packages\sentence_transformers\models\Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will 

In [12]:
all_embeddings

array([[-0.06946508,  0.02197973, -0.01702085, ..., -0.039938  ,
        -0.06433129, -0.0567485 ],
       [-0.01320128,  0.00849908, -0.01694448, ...,  0.00083938,
        -0.06353516, -0.00979694],
       [-0.07420228,  0.03161359, -0.02190242, ...,  0.03698989,
        -0.07184973, -0.05567618],
       ...,
       [-0.00855042,  0.02324842, -0.01468309, ..., -0.03881217,
        -0.04609815, -0.07335094],
       [-0.03033828,  0.01181995, -0.01519503, ..., -0.03026196,
        -0.048368  , -0.06836893],
       [-0.03776875,  0.00140323, -0.00231405, ..., -0.04760503,
        -0.06152563, -0.06940249]], dtype=float32)

In [14]:
temp_data = np.load('LaBSE_embeddings.npy')

In [15]:
temp_data

array([[-0.06946508,  0.02197973, -0.01702085, ..., -0.039938  ,
        -0.06433129, -0.0567485 ],
       [-0.01320128,  0.00849908, -0.01694448, ...,  0.00083938,
        -0.06353516, -0.00979694],
       [-0.07420228,  0.03161359, -0.02190242, ...,  0.03698989,
        -0.07184973, -0.05567618],
       ...,
       [-0.00855042,  0.02324842, -0.01468309, ..., -0.03881217,
        -0.04609815, -0.07335094],
       [-0.03033828,  0.01181995, -0.01519503, ..., -0.03026196,
        -0.048368  , -0.06836893],
       [-0.03776875,  0.00140323, -0.00231405, ..., -0.04760503,
        -0.06152563, -0.06940249]], dtype=float32)

### xlm-roberta-base

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModel.from_pretrained("xlm-roberta-base")

# Example long text
long_text = "Your long text goes here..."

# Define max length and sliding window step
max_length = 512
stride = 256

# Tokenize in a sliding window fashion
tokens = tokenizer(long_text, return_tensors='pt', truncation=True, padding=True, max_length=max_length, stride=stride, return_overflowing_tokens=True)

# Generate embeddings for each chunk
with torch.no_grad():
    embeddings = model(**tokens).last_hidden_state.mean(dim=1)

# Aggregate embeddings (e.g., average)
final_embedding = embeddings.mean(dim=0)

print(final_embedding.shape)
